In [1]:
import pandas as pd
import sys
import os
import numpy as np

In [2]:
df_credit = pd.read_csv(r"C:\Users\rtc32\OneDrive\바탕 화면\데이톤\Datasets\credit_card_balance.csv")

In [3]:


# 고객별 가장 오래된 신용카드 거래 개월 수 (음수)
earliest_card_months = df_credit.groupby('SK_ID_CURR')['MONTHS_BALANCE'].min().reset_index()
earliest_card_months.columns = ['SK_ID_CURR', 'FIRST_CARD_MONTH']

# 현재 기준으로 신용카드 거래기간 (개월 수, 양수로 변환)
earliest_card_months['CREDIT_CARD_HISTORY_MONTHS'] = -earliest_card_months['FIRST_CARD_MONTH']
earliest_card_months['CREDIT_CARD_HISTORY_YEARS'] = (earliest_card_months['CREDIT_CARD_HISTORY_MONTHS'] / 12).round(1)


In [4]:
earliest_card_months.columns

Index(['SK_ID_CURR', 'FIRST_CARD_MONTH', 'CREDIT_CARD_HISTORY_MONTHS',
       'CREDIT_CARD_HISTORY_YEARS'],
      dtype='object')

In [5]:
df2 = earliest_card_months[['SK_ID_CURR', 'CREDIT_CARD_HISTORY_YEARS']]

In [6]:
df_credit_kpi = pd.read_csv("credit_kpi2.csv")

In [7]:
df_credit_kpi3 = df_credit_kpi.merge(df2, on='SK_ID_CURR', how='left')

In [9]:
df_BB = pd.read_csv(r"C:\Users\rtc32\OneDrive\바탕 화면\데이톤\Datasets\bureau_balance.csv")
df_b = pd.read_csv(r"C:\Users\rtc32\OneDrive\바탕 화면\데이톤\Datasets\bureau.csv")

In [10]:
# 1. 각 대출 ID(SK_ID_BUREAU) 기준으로 가장 오래된 기록 찾기
bureau_bal_min_months = df_BB.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].min().reset_index()
bureau_bal_min_months.columns = ['SK_ID_BUREAU', 'FIRST_LOAN_MONTH']

# 2. bureau 테이블에서 고객 ID와 연결
bureau_with_customer = df_b[['SK_ID_BUREAU', 'SK_ID_CURR']]
bureau_bal_with_customer = pd.merge(bureau_bal_min_months, bureau_with_customer, on='SK_ID_BUREAU', how='left')

# 3. 고객 단위로 가장 오래된 대출 거래 찾기
customer_loan_history = bureau_bal_with_customer.groupby('SK_ID_CURR')['FIRST_LOAN_MONTH'].min().reset_index()
customer_loan_history['LOAN_HISTORY_MONTHS'] = -customer_loan_history['FIRST_LOAN_MONTH']  # 음수를 양수로
customer_loan_history['LOAN_HISTORY_YEARS'] = (customer_loan_history['LOAN_HISTORY_MONTHS'] / 12).round(1)


In [ ]:
customer_loan_history

In [11]:
df_credit_all = df_credit.merge(df_credit_kpi3, on='SK_ID_CURR', how='left')

In [12]:
df_credit_all.to_csv("credit_kpi_final.csv", index=False)
df_credit_kpi3.to_csv("credit_kpi3.csv", index=False)

In [13]:
earliest_card_months.describe()

,SK_ID_CURR,FIRST_CARD_MONTH,CREDIT_CARD_HISTORY_MONTHS,CREDIT_CARD_HISTORY_YEARS
count,103558.000000,103558.000000,103558.000000,103558.000000
mean,278381.457956,-37.534531,37.534531,3.128623
std,102779.519683,33.379821,33.379821,2.780987
min,100006.000000,-96.000000,1.000000,0.100000
25%,189536.250000,-75.000000,11.000000,0.900000
50%,278649.000000,-22.000000,22.000000,1.800000
75%,367690.000000,-11.000000,75.000000,6.200000
max,456250.000000,-1.000000,96.000000,8.000000


In [14]:
df_app = pd.read_csv(r"C:\Users\rtc32\OneDrive\바탕 화면\데이톤\Datasets\application_train.csv")

In [15]:
df_app = df_app[['SK_ID_CURR', 'TARGET']]

In [16]:
df_loan = customer_loan_history.merge(df_app, on='SK_ID_CURR', how='left')

In [17]:
df_loan.to_csv("customer_loan_history.csv", index=False)